In [2]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

#%%timeit -n1 -r1

['/device:CPU:0', '/device:GPU:0']


In [39]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

from glob import glob
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from tqdm import tqdm
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import layers
from tensorflow.keras import models

In [20]:
def get_image_labels():
#     columns_to_drop = [
#         'Follow-up #', 'Patient ID', 'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width', 'Height]',
#         'OriginalImagePixelSpacing[x', 'y]', 'Unnamed: 11'
#     ]
    
    columns_to_drop = [
        'Follow-up #', 'Patient ID', 'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width', 'Height]',
        'OriginalImagePixelSpacing[x', 'y]'
    ]
    
    df = pd.read_csv('C:\\Users\\m7irt\\OneDrive\\Desktop\\NIHCC ChestXray\\Data_Entry_2017.csv')
    df = df.drop(columns = columns_to_drop)
    
    image_label_dict = {}
    for row in tqdm(range(112120), desc = "Progress"):
        image_name = df.iat[row, 0]
        disease_name = df.iat[row, 1]
        disease_name = disease_name.split("|")
        image_label_dict[image_name] = disease_name
        
    return image_label_dict

In [21]:
def get_image_paths():
    image_paths = {}
    paths = glob(os.path.join('images', '*.png'))
    for path in tqdm(paths, desc = "Progress"):
        image_name = os.path.basename(path)
        image_name = image_name.strip(".png")
        image_paths[image_name] = path
    
    return image_paths

In [40]:
def labels_to_array(image_labels):
    labels = []
    paths = glob(os.path.join('images', '*.png'))
    for path in paths:
        image_name = os.path.basename(path)
        #image_name = image_name.strip(".png")
        labels.append(image_labels[image_name])
        
    return labels

In [50]:
def get_meta_information_dataframe():
    column_names = ["Image_Name", "Image_Path", "Binary_Image_Labels", "Image_Labels", "Image_Labels_List"]
    meta_data = pd.DataFrame(columns = column_names)
    print(meta_data)
    return meta_data

In [51]:
meta_information_dataframe = get_meta_information_dataframe()

Empty DataFrame
Columns: [Image_Name, Image_Path, Binary_Image_Labels, Image_Labels, Image_Labels_List]
Index: []


In [22]:
image_labels = get_image_labels()

Progress: 100%|██████████| 112120/112120 [00:05<00:00, 21533.79it/s]

Cardiomegaly|Emphysema


In [26]:
print(image_labels["00000001_000.png"])
print(image_labels["00000001_001.png"])
print(image_labels["00000001_002.png"])

['Cardiomegaly']
['Cardiomegaly', 'Emphysema']
['Cardiomegaly', 'Effusion']


In [41]:
labels_array = labels_to_array(image_labels)

In [42]:
print(labels_array[0:20])

[['Cardiomegaly'], ['Cardiomegaly', 'Emphysema'], ['Cardiomegaly', 'Effusion'], ['No Finding'], ['Hernia'], ['Hernia'], ['Hernia'], ['Hernia', 'Infiltration'], ['Hernia'], ['Hernia'], ['Hernia'], ['Hernia'], ['Mass', 'Nodule'], ['No Finding'], ['No Finding'], ['No Finding'], ['No Finding'], ['No Finding'], ['No Finding'], ['Infiltration']]


In [45]:
binarizer = MultiLabelBinarizer()
one_hot = binarizer.fit_transform(labels_array)
print(one_hot[0:20])
print(binarizer.classes_)

[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]]
['Atelectasis' 'Cardiomegaly' 'Consolidation' 'Edema' 'Effusion'
 'Emphysema' 'Fibrosis' 'Hernia' 'Infiltration' 'Mass' 'No Finding'
 'Nodule' 'Pleural_Thickening' 'Pneumonia' 'Pneumothorax']


In [19]:
image_paths = get_image_paths()

Progress: 100%|██████████| 112120/112120 [00:00<00:00, 236487.04it/s]


In [3]:
resnet50_model = ResNet50(
    include_top = False,
    weights = 'imagenet',
    input_tensor = None,
    input_shape = None,
    pooling = None,
    classes = 15,
    classifier_activation = "softmax"
)

ValueError: If using `weights` as `"imagenet"` with `include_top` as true, `classes` should be 1000